# Complementaria Repaso Quiz 3 - Miércoles 12:30

In [1]:
import numpy as np
import pandas as pd
from jmarkov.mdp.dtmdp import dtmdp

In [2]:
# Matrices de transición (orden de estados: T, N)

P_A = np.array([
    [0.75, 0.25],
    [0.60, 0.40]
])

P_C = np.array([
    [0.55, 0.45],
    [0.50, 0.50]
])

P_K = np.array([
    [0.80, 0.20],
    [0.55, 0.45]
])

P_D = np.array([
    [0.55, 0.45],
    [0.10, 0.90]
])

# Ingresos por estado (USD por mes)
ingresos = {
    'T': 140_000,
    'N': 40_000
}

# Costos por acción (USD por mes)
costos = {
    'A': -20_000,   # Nuevo álbum
    'C': -10_000,   # Colaboración
    'K': -25_000,   # Concierto
    'D': 0          # Descanso
}

## Parte A: Implementación del MDP

In [12]:
# Variable de estado
estados = np.array(["T","N"], dtype=str)

# Acciones

acciones = np.array(["A","C","K","D"])

# Matrices de probabilidad

matrices = {}
for a in acciones:
    if a == "A":
        matrices[a] = P_A
    elif a == "C":
        matrices[a] = P_C
    elif a == "K":
        matrices[a] = P_K
    elif a == "D":
        matrices[a] = P_D

# Retornos (ingresos - costos)

retornos = np.zeros((len(estados),len(acciones)))

for i, s in enumerate(estados):
    for j, a in enumerate(acciones):
        retornos[i,j] = ingresos[s] + costos[a]

retornos_df = pd.DataFrame(retornos, index=estados, columns=acciones)

minimize = False

# MDP

beta = 0.95

mdp = dtmdp(estados, acciones, matrices, retornos, beta)

# Valores optimos para cada estado

value_functions, optimal_policy = mdp.solve(0, minimize=minimize, method="value_iteration")

print(value_functions)

print(optimal_policy)

# valor esperado de la politica

mdp.expected_policy_value(value_functions, optimal_policy)

# Matriz de transicion inducida de la politica optima

matrizPO = mdp.policy_transition_matrix(optimal_policy)

matrizPO_df = pd.DataFrame(matrizPO, index = estados, columns = estados)

matrizPO_df

[1854320.98765414 1737037.03703685]
{np.str_('T'): np.str_('K'), np.str_('N'): np.str_('A')}


,T,N
T,0.8,0.2
N,0.6,0.4


## Parte B: Simulación de Monte Carlo

In [25]:
escenarios = 1000 # simulaciones
horizon = 20 # meses
estado_inicial = "N"
# retornos_df y matrizpo_df ya

np.random.seed(0)

Y = np.zeros((horizon, escenarios))

indicadores_estados = {s: np.zeros((horizon, escenarios)) for s in estados}

for j in range(escenarios):

    estado = estado_inicial

    visista_evento = 0

    for t in range(horizon):

        accion = str(optimal_policy[estado])

        Y[t,j] = retornos_df.loc[estado, accion]

        if estado == "T":
            indicadores_estados["T"][t,j] = 1

        estado = np.random.choice(estados, p=matrizPO_df.loc[estado,:].values)


In [29]:
# Ingreso acumulado esperado durante las ultimas 5 meses - Caso 3

respuesta_1 = Y[-5:,:].sum(axis=0).mean()

# En que porcentaje de meses baddoggy se encuentra en estado T - Caso 4

respuesta_2 = indicadores_estados["T"].mean() * 100

print(respuesta_1)
print(respuesta_2)

459100.0
70.62
